In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys

sys.path.append(os.path.abspath(os.pardir))

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
# COUNTRY_NAME = 'MLI Mali'
COUNTRY_NAME = os.environ.get("COUNTRY_NAME")
SPOKEN_LANGUAGE = "French"

table_column_name = "AnneeCodeDebut"

print(COUNTRY_NAME)

BFA Burkina Faso


In [4]:
from general_config import COUNTRY_NAMES_LIST

# validate COUNTRY_NAME
if COUNTRY_NAME in COUNTRY_NAMES_LIST:
    print('country name OK')

country name OK


In [5]:
from llm_setup import *

# Get legal documents from existing paperless Postgres

In [6]:
df, documents = get_country_legal_docs(COUNTRY_NAME)

/home/andrei/Ferdi_LLM/postgres_connection.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


# Set the embeddings model and create a vectorized DB for the extracted texts

In [9]:
embed_model = get_voyage_ai_embedding_model()

In [10]:
vector_index = get_vector_db_from_documents(documents, COUNTRY_NAME, embed_model, chunk_size = int(1024))

loading from local


In [14]:
doc_titles = pd.Series(list(set([v.get('title') for k,v in vector_index.vector_store.to_dict()['metadata_dict'].items()])))
doc_titles[0]

'Loi n°2015-036 (26.06.2015) Code minier 2015 (Alain Charlet)'

# Set the llm
the llm will be used to augment the retrieved data from the vector_index

In [15]:
llm = get_open_ai_llm()

# RAG to extract the Annee Code Debut
 - create a vector engine using the above created vector database and the title of the document
 - using the llm to augment the retrieved response, and generate the response according to the prompt

In [16]:
def get_anee_debut(vector_index, doc_name, llm):
    vector_query_engine = create_vector_engine(vector_index, doc_name, llm)

    response = vector_query_engine.query(
        f"""
        You are a helpful legal assistant, who can read legal documents and understand them.
        Consider the title and the contents of the document with this title '{doc_name}'. In which year did this document become valid?
        If you don't know the answer or it cannot be retrieved from the content or the title of the text, reply with 'None'.
        """
    )

    print(doc_name, len(response.source_nodes))
    print([node.score for node in response.source_nodes])
    print([node.text for node in response.source_nodes])

    return response.response

### Test the above function

In [18]:
response = get_anee_debut(
    vector_index=vector_index,
    doc_name = doc_titles[0],
    llm=llm) # doc_titles[6]

print(response)

Loi n°2015-036 (26.06.2015) Code minier 2015 (Alain Charlet) 2
[0.890625, 0.8359375]
['langue française.\n\n\n\n\n83\nArticle 214 :\n\nLa présente loi abroge toutes dispositions antérieures contraires,\nnotamment la loi n°031-2003/AN du 08 mai 2003 portant code minier au\nBurkina Faso.\n\nArticle 215 :\n\nLa présente loi sera exécutée comme loi de l’Etat.\n\n\n\n\nAinsi fait et délibéré en séance publique\nà Ouagadougou, le 26 juin 2015\n\n\n\n\n84', "et des\ncollectivités ainsi que dans les zones faisant l'objet d'un titre minier ou\nd'une autorisation d'exploitation dans le respect des dispositions des\narticles 119 et 120 du présent code.\n\nL'autorisation de prospection ne confère à son titulaire aucun droit pour\nl'obtention subséquente d'un titre minier ou d'une autre autorisation.\n\nArticle 68 :\n\nL'autorisation de prospection est valable pour un an à compter de sa date\nde délivrance. Elle est renouvelable une fois par décision de l'autorité qui l'a\ndélivrée et dans les même

In [21]:
large_categs = list()

for doc in doc_titles:
    try:
        categ = get_anee_debut(vector_index=vector_index, doc_name = doc, llm = llm)
    except Exception as e:
        print(e)

        categ = "non_determined"

    large_categs.append((doc, categ))

    

Loi n°2015-036 (26.06.2015) Code minier 2015 (Alain Charlet) 2
[0.890625, 0.8359375]
['langue française.\n\n\n\n\n83\nArticle 214 :\n\nLa présente loi abroge toutes dispositions antérieures contraires,\nnotamment la loi n°031-2003/AN du 08 mai 2003 portant code minier au\nBurkina Faso.\n\nArticle 215 :\n\nLa présente loi sera exécutée comme loi de l’Etat.\n\n\n\n\nAinsi fait et délibéré en séance publique\nà Ouagadougou, le 26 juin 2015\n\n\n\n\n84', "et des\ncollectivités ainsi que dans les zones faisant l'objet d'un titre minier ou\nd'une autorisation d'exploitation dans le respect des dispositions des\narticles 119 et 120 du présent code.\n\nL'autorisation de prospection ne confère à son titulaire aucun droit pour\nl'obtention subséquente d'un titre minier ou d'une autre autorisation.\n\nArticle 68 :\n\nL'autorisation de prospection est valable pour un an à compter de sa date\nde délivrance. Elle est renouvelable une fois par décision de l'autorité qui l'a\ndélivrée et dans les même

In [22]:
df_AnneeCodeDebut = pd.DataFrame.from_dict(dict(large_categs), orient='index', columns=['AnneeCodeDebut'])

df_AnneeCodeDebut.to_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv", index=True)

df_AnneeCodeDebut

,AnneeCodeDebut
Loi n°2015-036 (26.06.2015) Code minier 2015 (Alain Charlet),2015
Loi n°1995-062 (14.12.1995) Code des investissements 1995 (LégiBurkina),1995
Décret n°2004-580 (15.12.2004) Eau 2004 (Faolex),2004
Loi n°2003-062 (13.12.2003) Loi de finances 2004 (LégiBurkina),2004
Loi n°1994-060 (20.12.1994) Loi de finances 1995 (LégiBurkina),1995
...,...
Loi n°2010-007 (29.01.2010) Modification Code des investissements 2010 (DGI),2010
Loi n°1965-006 (26.05.1965) Code des impôts directs et indirects 1996 (LégiBurkina),1965
Décret n°2002-267 (25.07.2002) Convention minière type 2002 (LégiBurkina),2002
img_Loi n°2020-031 (09.07.2020) Loi de finances rectificative 2020 (Droit-Afrique),2020


## Load existing categories

In [23]:
# COUNTRY_NAME = 'MLI Mali'

large_categs = pd.read_csv(f"output/{table_column_name}_{COUNTRY_NAME}.csv")
large_categs = list(list(large_categs.set_index(['Unnamed: 0']).to_dict().values())[0].items())
# large_categs

## Update column in table

In [24]:
from postgres_connection import bulk_update_textes

In [25]:
# bulk_update_employees(textes_to_update = large_categs[0:1], )

bulk_update_textes(
    textes_to_update = large_categs, 
    column_name = table_column_name, 
    COUNTRY_NAME=COUNTRY_NAME)

Loi n°2015-036 (26.06.2015) Code minier 2015 (Alain Charlet) column AnneeCodeDebut updated to 2015.
Loi n°1995-062 (14.12.1995) Code des investissements 1995 (LégiBurkina) column AnneeCodeDebut updated to 1995.
Décret n°2004-580 (15.12.2004) Eau 2004 (Faolex) column AnneeCodeDebut updated to 2004.
Loi n°2003-062 (13.12.2003) Loi de finances 2004 (LégiBurkina) column AnneeCodeDebut updated to 2004.
Loi n°1994-060 (20.12.1994) Loi de finances 1995 (LégiBurkina) column AnneeCodeDebut updated to 1995.
Loi n°2016-040 (15.12.2016) Loi de finances 2017 (Droit-Afrique) column AnneeCodeDebut updated to 2017.
img_Décret n°2017-023 (23.01.2017) Taxes et redevances minières 2017 (African Mining Legislation Atlas) column AnneeCodeDebut updated to 2017.
Loi n°1996-048 (27.11.1996) Modification BIC BNC 1996 (LégiBurkina) column AnneeCodeDebut updated to 1997.
Loi n°2017-052 (27.11.2017) Loi de finances 2018 (Assemblée Nationale) column AnneeCodeDebut updated to 2018.
Loi n°1993-037 (10.12.1993) Loi d